In [1]:
from os.path import join
import os
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
from typing import Literal

# Create DataFrames for each ID type
We want to create three lookup tables from the CSV files mapping PIDs to OpenAlex IDs for the resources in OpenAlex. We can do so by creating three different DataFrames, one for each ID type (doi, pmid, pmcid). Each DataFrame will have two columns: the first one will contain the PID of the ID type (doi, pmid, pmcid) and the second one will contain the OpenAlex ID of the bibliographic resource it is associated with. We can later use these DataFrames to create the corresponding tables in the database.
As the task is memory-intensive, we need to create the ID-type DataFrames one at a time. We define a function that takes in input the prefix (doi|pmid|pmcid) needed to select the rows from the input CSV files and the input directory; it returns the corresponding a single DataFrame, where each line corresponds to a bibliographic resource and stores one PID of the ID type specified as a parameter and the OpenAlex ID of the bibliographic resource it is associated with. (Only takes into consideration the IDs that can be associated with single bibliographic resources: doi, pmid, pmcid; isbn and issn are excluded!).

In [30]:
# from typing import Literal
#
# def create_id_df(id_type: Literal['doi','pmid','pmcid'], directory_path:str):
#     """
#     Create a DataFrame for a given ID type (doi, pmid, pmcid) from the CSV files in the input directory.
#     :param id_type: the ID type (doi, pmid, pmcid)
#     :param directory_path: the path to the directory containing the CSV files
#     :return: a DataFrame containing the PIDs of the ID type specified as a parameter and the OpenAlex IDs of the
#     bibliographic resources they are associated with
#     """
#     try:
#         assert id_type in ['doi','pmid','pmcid']
#         id_df = pd.DataFrame(columns=['supported_id', 'openalex_id'])
#         for root, dirs, files in os.walk(directory_path):
#             for file in tqdm(files):
#                 if file.endswith('.csv'):
#                     file_path = os.path.join(root, file)
#                     current_df = pd.read_csv(file_path)
#
#                     # Determine the prefix and concatenate with the corresponding DataFrame
#                     df_id = current_df[current_df['supported_id'].str.startswith(id_type)]
#                     id_df = pd.concat([id_df, df_id], ignore_index=True)
#         return id_df
#     except AssertionError:
#         print('The ID type must be one of the following: doi, pmid, pmcid')
#         return None

## Still to slow...
Let's try another approach. In the following function we first read each csv file line by line, then we append relevant rows to a list. Then we create a DataFrame from the list of rows and return the DataFrame.

In [31]:
# import pandas as pd
# import os
# import csv
# from tqdm import tqdm
# from typing import Literal
#
# def create_id_df(id_type: Literal['doi', 'pmid', 'pmcid'], directory_path: str):
#     """
#     Create a DataFrame for a given ID type (doi, pmid, pmcid) from the CSV files in the input directory.
#     :param id_type: the ID type (doi, pmid, pmcid)
#     :param directory_path: the path to the directory containing the CSV files
#     :return: a DataFrame containing the PIDs of the ID type specified as a parameter and the OpenAlex IDs of the
#     bibliographic resources they are associated with
#     """
#     try:
#         assert id_type in ['doi', 'pmid', 'pmcid']
#         id_data = []
#         for root, dirs, files in os.walk(directory_path):
#             for file in tqdm(files):
#                 if file.endswith('.csv'):
#                     file_path = os.path.join(root, file)
#                     with open(file_path, 'r', encoding='utf-8') as csv_file:
#                         reader = csv.DictReader(csv_file, dialect='unix')
#                         for row in reader:
#                             if row['supported_id'].startswith(id_type):
#                                 id_data.append(row)
#
#         id_df = pd.DataFrame(id_data, columns=['supported_id', 'openalex_id'])
#         return id_df
#     except AssertionError:
#         print('The ID type must be one of the following: doi, pmid, pmcid')
#         return None

## Still too slow (x2)...
This still takes forever to run. The RAM saturates pretty soon, and though it is a bit faster at the beginning, before even finishing building the list it incurs in a MemoryError.
## Change the function for better performance
Instead of creating a single DataFrame for all the files, we can create a DataFrame for each file and append it to the database table straight away. In this way, we don't need to store all the DataFrames in memory at the same time, thus avoiding to saturate the RAM and force the system to use the swap memory.

The following function creates a table (if it doesn't already exist) and names it with the name of the ID type passed as a parameter (one among "doi", "pmid" and "pmcid"). Then, for each csv file in the input directory, the file is converted to a pandas DataFrame and then appended to the database table. The DataFrames, each of which corresponds to a single file,are appended one at a time.

In [2]:
OA_WORK_OUTPUT_FOLDER_PATH = join('D:/oa_work_tables')

directory_path = OA_WORK_OUTPUT_FOLDER_PATH  # Path to the directory containing the CSV files

In [3]:
def create_id_db_table(inp_dir:str, db_path:str, id_type:Literal['doi', 'pmid', 'pmcid', 'wikidata', 'issn'], entity_type: Literal['work', 'source'])-> None:

    table_name = f'{entity_type.capitalize()}s{id_type.capitalize()}'
    start_time = time.time()
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if cursor.fetchone():
            raise ValueError(f"Table {table_name} already exists")

        for root, dirs, files in os.walk(inp_dir):
            for file in tqdm(files):
                if file.endswith('.csv'):
                    csv_path = os.path.join(root, file)
                    file_df = pd.read_csv(csv_path)  # Read the CSV file into a DataFrame

                    # Select only the rows with the ID type specified as a parameter and create a new DataFrame
                    id_df = file_df[file_df['supported_id'].str.startswith(id_type)]

                    # Append the DataFrame's rows to the existing table in the database
                    id_df.to_sql(table_name, conn, if_exists='append', index=False)

    print(f"Creating the database table for {id_type.upper()}s took {(time.time()-start_time)/60} minutes")


In [34]:
# create database table for DOIs
create_id_db_table(directory_path, 'oa_ids_tables.db', 'doi', 'work')

0it [00:00, ?it/s]
100%|██████████| 41/41 [01:00<00:00,  1.47s/it]

Creating the database table for DOIs took 9.362805151939392 minutes


In [35]:
# create database table for PMIDs
create_id_db_table(directory_path, 'oa_ids_tables.db', 'pmid', 'work')

0it [00:00, ?it/s]
100%|██████████| 41/41 [00:43<00:00,  1.06s/it]

Creating the database table for PMIDs took 6.477690362930298 minutes


In [36]:
# create database table for PMCIDs
create_id_db_table(directory_path,'oa_ids_tables.db','pmcid', 'work')

0it [00:00, ?it/s]
100%|██████████| 41/41 [01:23<00:00,  2.03s/it]

Creating the database table for PMCIDs took 6.504606902599335 minutes


In [4]:
source_tables_dir = 'D:/oa_source_tables'

In [5]:
# create database table for Wikidata Q-IDs (from Sources)
create_id_db_table(source_tables_dir,'oa_ids_tables.db','wikidata', 'source')

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  6.36it/s]

Creating the database table for WIKIDATAs took 0.00730438232421875 minutes


In [6]:
# create database table for ISSNs (from Sources)
create_id_db_table(source_tables_dir,'oa_ids_tables.db','issn', 'source')

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  4.49it/s]

Creating the database table for ISSNs took 0.008862519264221191 minutes


In [7]:
# Getting the table names from the database

# Connect to the SQLite database
conn = sqlite3.connect('oa_ids_tables.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the query to retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
table_names = cursor.fetchall()

# Print the table names
for name in table_names:
    print(name[0])

# Close the cursor and the database connection
cursor.close()
conn.close()


WorksDoi
WorksPmid
WorksPmcid
SourcesWikidata
SourcesIssn


### Creating table's indexes
In order to speed up the queries, we can create indexes on the columns of the tables. In particular, we can create indexes on the `supported_id` column of each table, since this is the column we will use to query the tables. Without indexes, the queries will be very slow, since the database will have to scan the entire table to find the rows that match the query. With indexes, the database will be able to find the rows that match the query much faster, since it will only have to scan the index, which is much smaller than the table itself.

Creating the indexes on the tables takes a while, but not too much: creating the index for the DOI table took about 8 minute. Comment out the following code if you want to create the indexes.

In [8]:
import sqlite3 as sql

conn = sql.connect('oa_ids_tables.db')
cursor = conn.cursor()
# start_time_idx_doi = time.time()
# cursor.execute("CREATE INDEX idx_doi_works ON WorksDoi(supported_id)")
# print(f"Creating the index on the DOI table took {(time.time()-start_time_idx_doi)/60} minutes")
# start_time_idx_pmid = time.time()
# cursor.execute("CREATE INDEX idx_pmid_works ON WorksPmid(supported_id)")
# print(f"Creating the index on the PMID table took {(time.time()-start_time_idx_pmid)/60} minutes")
# start_time_idx_pmcid = time.time()
# cursor.execute("CREATE INDEX idx_pmcid_works ON WorksPmcid(supported_id)")
# print(f"Creating the index on the PMCID table took {(time.time()-start_time_idx_pmcid)/60} minutes")
start_time_idx_pmcid = time.time()
cursor.execute("CREATE INDEX idx_wikidata_sources ON SourcesWikidata(supported_id)")
print(f"Creating the index on the Wikidata table took {(time.time()-start_time_idx_pmcid)/60} minutes")
start_time_idx_pmcid = time.time()
cursor.execute("CREATE INDEX idx_issn_sources ON SourcesIssn(supported_id)")
print(f"Creating the index on the ISSN table took {(time.time()-start_time_idx_pmcid)/60} minutes")
cursor.close()
conn.close()


Creating the index on the Wikidata table took 0.0006640275319417317 minutes
Creating the index on the ISSN table took 0.0018349250157674154 minutes


We can see if the indexes have been created and how they are named by executing the following query:

In [9]:
conn = sql.connect('oa_ids_tables.db')
cursor = conn.cursor()

query = "SELECT name, tbl_name FROM sqlite_master WHERE type = 'index';"
cursor.execute(query)
print(cursor.fetchall())
cursor.close()
conn.close()


[('idx_doi_works', 'WorksDoi'), ('idx_pmid_works', 'WorksPmid'), ('idx_pmcid_works', 'WorksPmcid'), ('idx_wikidata_sources', 'SourcesWikidata'), ('idx_issn_sources', 'SourcesIssn')]


## Stats on the Mapping

In [2]:
import os
import csv
from tqdm import tqdm

# Path to the directory containing the CSV files
directory = 'D:/omid_openalex_mapping'

# Variables to store the statistics
omid_set = set()
openalex_id_set = set()
multi_mapped_omids_count = 0
multi_mapped_oaid_by_type = {}

# Iterate over each CSV file
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            csv_reader = csv.reader(file, delimiter=',', dialect='unix')

            # Skip the header line
            next(csv_reader)

            for row in csv_reader:
                omid = row[0]
                openalex_ids = row[1].split()
                oaid_count = len(openalex_ids)

                # Update omid count
                omid_set.add(omid)

                # Update openalex_id count
                openalex_id_set.update(openalex_ids)

                # Update multi-mapped OMIDs count
                if oaid_count > 1:
                    multi_mapped_omids_count += 1

                # Update multi-mapped OMID count by type
                    oaid_type = row[2]
                    multi_mapped_oaid_by_type[oaid_type] = multi_mapped_oaid_by_type.get(oaid_type, 0) + 1

# Extracting statistics
omid_count = len(omid_set)
openalex_id_count = len(openalex_id_set)

# Printing the statistics
print(f"1) Number of unique omids: {omid_count}")
print(f"2) Number of unique openalex_ids: {openalex_id_count}")
print(f"3a) Number of multi-mapped OMIDs: {multi_mapped_omids_count}")
print("3b) Number of multi-mapped OMIDs for each value of 'type':")
for oaid_type, count in multi_mapped_oaid_by_type.items():
    print(f"{oaid_type}: {count}")


100%|██████████| 22195/22195 [05:52<00:00, 62.94it/s]  

1) Number of unique omids: 74567082
2) Number of unique openalex_ids: 74586242
3a) Number of multi-mapped oaids: 17897
3b) Number of multi-mapped oaids for each value of 'type':
reference book: 69
series: 1
standard: 7
book series: 1
journal: 241
journal article: 10001
book: 7247
proceedings article: 106
reference entry: 108
book chapter: 95
report: 12
web content: 4
proceedings: 4
dataset: 1


In [ ]:
# Getting the total number of OAIDs in the Work folder of the OpenAlex dump
import os
import csv
from tqdm import tqdm

# Path to the directory containing the CSV files
directory = 'D:/oa_work_tables'

# Variables to store the statistics
oaids_set = set()
for root, dirs, files in os.walk(directory):
            for file in tqdm(files):
                if file.endswith('.csv'):
                    csv_path = os.path.join(root, file)
                    with open(csv_path, encoding='utf-8') as file:
                        csv_reader = csv.reader(file, delimiter=',', dialect='unix')

                        # Skip the header line
                        next(csv_reader)

                        for row in csv_reader:
                            oaids_set.add(row[1])

print(f"Number of OAIDs in the Work folder of the OpenAlex dump: {len(oaids_set)}")